In [41]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn import model_selection
import pickle


import pandas as pd
import numpy as np

In [42]:
df = pd.read_csv("E:/research/Spacematch/formal research/feedback_89.csv") #import the data-points
# df.count()

In [43]:
y = df.sensations_temperature
X = df.drop(["user_id","place_id", "created_at","sensations_temperature","preferences_temperature"], axis = 1)  #Get X and y

In [44]:
# X

In [45]:
rt_data = pd.read_csv("E:/research/Spacematch/formal research/real_time_room_data.csv")
rt_data = rt_data.drop(["room_id"], axis = 1)
X = X.append(rt_data)

def createNewTime(row):
    if row['Time'] <= 12:
        return 1
    else:
        return 2
    
X['New Time'] = X.apply(createNewTime, axis=1)
X = X.drop(['Time'], axis = 1)
rt_data = X.iloc[-4:]
X = X.iloc[:74]

In [46]:
rt_data

,clothing_level,indoor_temp,indoor_humidity,outdoor_temp,outdoor_humidity,New Time
0,1,69.03,49.55,88.0,55.0,2
1,1,70.08,48.02,88.0,55.0,2
2,1,73.71,46.40,88.0,55.0,2
3,1,76.60,43.16,88.0,55.0,2


In [47]:
# param_test = {"C"               :np.logspace(-1, 3, num=10, endpoint=True, base=10, dtype=None, axis=0),
#               'gamma'           :np.logspace(-1, 3, num=10, endpoint=True, base=10, dtype=None, axis=0)
#               "kernel"          :['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
#               } # Optimize the parameters
# for i in range (2,9,1):
#     gsearch = GridSearchCV(estimator = LogisticRegression(
#         multi_class = 'multinomial', max_iter = 10000, random_state = 10), param_grid = param_test, cv = i)
#     gsearch.fit(X, y)
#     print(i)
#     solver_p    = gsearch.best_params_["solver"]
#     C_p         = gsearch.best_params_["C"]
#     para = (solver_p, C_p, gsearch.best_score_)
#     print(para)
#     loo = LeaveOneOut() # test the accuracy with leaveoneout
#     y_true = []
#     y_pred = []
#     for train_index, test_index in loo.split(X): # Split in X
#         X_train, X_test = X.values[train_index], X.values[test_index]
#         y_train, y_test = y.values[train_index], y.values[test_index]
#         lr = LogisticRegression(multi_class = 'multinomial', max_iter = 10000, random_state = 10,
#                                 solver = solver_p, C = C_p,)
#         lr = lr.fit(X_train, y_train)
#         predict_y = lr.predict(X_test)
#         y_true.append(y_test[0])
#         y_pred.append(predict_y[0])
#     accuracy = accuracy_score(y_true, y_pred)
#     print("accuracy is:", accuracy)
#     lr = LogisticRegression(multi_class = 'multinomial', max_iter = 10000, random_state = 10
#                             , solver = solver_p, C = C_p)
#     lr.fit(X,y)
#     print(lr.predict(rt_data))
#     rt_prepro = lr.predict_proba(rt_data)
#     print("probability : \n", rt_prepro)

In [48]:
'linear', 'poly', 'rbf', 'sigmoid'

('linear', 'poly', 'rbf', 'sigmoid')

In [51]:
param_test = {"C"               :0.2 ** np.linspace(1,10,10),
              'gamma'           :9 ** np.linspace(1,10,10),
              "kernel"          :['linear', 'rbf', 'sigmoid']
              } # Optimize the parameters
gsearch = GridSearchCV(estimator = SVC(probability=True, random_state = 10),param_grid = param_test, cv = 4)
gsearch.fit(X,y)
gsearch.predict(rt_data)
print(gsearch.predict_proba(rt_data))
print(gsearch.best_params_)


[[9.92216571e-01 2.93718322e-04 7.48971050e-03]
 [9.79490008e-01 1.56983804e-03 1.89401541e-02]
 [9.02650188e-01 1.76221692e-02 7.97276432e-02]
 [4.83529369e-01 1.70772940e-01 3.45697691e-01]]
{'C': 0.2, 'gamma': 9.0, 'kernel': 'linear'}


In [50]:
param_test = {"C"               :0.1 ** np.linspace(0,5,10),
              'gamma'           :10 ** np.linspace(0,5,10),
              "kernel"          :['linear', 'rbf', 'sigmoid']
              } # Optimize the parameters

cv_param = []
C_param = []
gamma_param = []
kernel_param = []

accuracy_param = []
loo_accuracy_param = []

room_1_neutral = []
room_2_neutral = []
room_3_neutral = []
room_4_neutral = []

for i in range (2,9,1):
    gsearch = GridSearchCV(estimator = SVC(probability=True, random_state = 10),
            param_grid = param_test, cv = i)
    gsearch.fit(X, y)
    C_p           = gsearch.best_params_["C"]
    gamma_p       = gsearch.best_params_["gamma"]
    kernel_p             = gsearch.best_params_["kernel"]
    
    cv_param.append(i)
    C_param.append(C_p)
    gamma_param.append(gamma_p)
    kernel_param.append(kernel_p)

    accuracy_param.append(gsearch.best_score_)
    
    loo = LeaveOneOut() # test the accuracy with leaveoneout
    y_true = []
    y_pred = []
    for train_index, test_index in loo.split(X): # Split in X
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y.values[train_index], y.values[test_index]
        svc = SVC(C = C_p, gamma = gamma_p, kernel = kernel_p, probability=True, random_state = 10)
        svc = svc.fit(X_train, y_train) # train model
        predict_y = svc.predict(X_test)
        y_true.append(y_test[0])
        y_pred.append(predict_y[0])
    accuracy = accuracy_score(y_true, y_pred)
    loo_accuracy_param.append(accuracy)
    
    svc = SVC(C = C_p, gamma = gamma_p, kernel = kernel_p, probability=True, random_state = 10)
    svc = svc.fit(X, y)
    
    
    rt_prepro = svc.predict_proba(rt_data)
    room_1_neutral.append(rt_prepro[0][1])
    room_2_neutral.append(rt_prepro[1][1])
    room_3_neutral.append(rt_prepro[2][1])
    room_4_neutral.append(rt_prepro[3][1])


KeyboardInterrupt: 

In [ ]:
result = pd.DataFrame(list(zip(cv_param, C_param, gamma_param, kernel_param, accuracy_param, loo_accuracy_param, 
                               room_1_neutral, room_2_neutral, room_3_neutral, room_4_neutral)),
                      columns = ['cv', 'C', 'gamma', 'kernel', 'accuracy', 'loo_accuracy', 
                                 'room_1', 'room_2', 'room_3', 'room_4'])

In [ ]:
result.to_csv('E:/research/Spacematch/formal research/feedback_89_result_svm.csv')

In [ ]:
# solver_p    = gsearch.best_params_["solver"]
# C_p         = gsearch.best_params_["C"]
# para = (solver_p, C_p, gsearch.best_score_)
# print(para)

In [ ]:
# loo = LeaveOneOut() # test the accuracy with leaveoneout
# y_true = []
# y_pred = []
# for train_index, test_index in loo.split(X): # Split in X
#     X_train, X_test = X.values[train_index], X.values[test_index]
#     y_train, y_test = y.values[train_index], y.values[test_index]
#     lr = LogisticRegression(multi_class = 'multinomial', max_iter = 5000, random_state = 10
#                             , solver = solver_p, C = C_p)
#     lr = lr.fit(X_train, y_train) # train model
#     predict_y = lr.predict(X_test)
#     y_true.append(y_test[0])
#     y_pred.append(predict_y[0])
# print(y_true)
# print(y_pred)
# accuracy = accuracy_score(y_true, y_pred)
# print("accuracy is:", accuracy)

In [ ]:
# # import the realtime room data
# rt_data = pd.read_csv("E:/research/Spacematch/formal research/real_time_room_data.csv")
# rt_data = rt_data.drop(["room_id"], axis = 1)
# rt_data.head()

In [ ]:
# lr = LogisticRegression(multi_class = 'multinomial', max_iter = 5000, random_state = 10, solver = solver_p, C = C_p)
# lr.fit(X,y)
# lr.get_params()

In [ ]:
# print(lr.predict(rt_data))
# rt_prepro = lr.predict_proba(rt_data)
# print("probability : \n", rt_prepro)